# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import json
import csv
import numpy as np
import pandas as pd
# import pkg_resources
import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from sklearn.preprocessing import StandardScaler

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.33.0


In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-courtlin",subscription_id="bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8", resource_group="udacity-nano-degree")
exp = Experiment(workspace=ws, name="stroke-prediction")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity-courtlin
Azure region: eastus2
Subscription id: bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8
Resource group: udacity-nano-degree


In [3]:
from azureml.core import Datastore
from azureml.core.compute import ComputeTarget, AmlCompute

datastore = ws.get_default_datastore()
    
cpu_cluster_name = "udacitycapstone"
if not cpu_cluster_name in ws.compute_targets :
    print('creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",# for GPU, use "STANDARD_NC6",
    max_nodes=5, vm_priority='lowpriority',min_nodes=0, idle_seconds_before_scaledown=300)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True,min_node_count=None, timeout_in_minutes=20)

    # Show the result
    print(cpu_cluster.get_status().serialize())

cpu_cluster = ws.compute_targets[cpu_cluster_name]

creating a new compute target...
InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2021-09-23T05:00:38.628000+00:00', 'errors': None, 'creationTime': '2021-09-23T05:00:38.240411+00:00', 'modifiedTime': '2021-09-23T05:00:53.861186+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 5, 'nodeIdleTimeBeforeScaleDown': 'PT300S'}, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

Overview of data used

Stroke is the 2nd leading cause of death globally, responsible for approximately 11% of total deaths, according to the WHO.
This dataset is used to predict whether a patient is likely to get stroke based on the input parameters like gender, age, various diseases, and smoking status.
The data contains 5110 observations with 12 attributes.
This will be a binary classification task to predict whether a patient is likely to have a stroke or not. 


In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-stroke'

experiment=Experiment(ws, experiment_name)

In [5]:
#Access the external data needed for the project that has been downloaded from Kaggle and uploaded to the Azure workspace
#This dataset, title Stroke Prediction Dataset, was downloaded from Kaggle for educational use. URL: https://www.kaggle.com/fedesoriano/stroke-prediction-dataset?select=healthcare-dataset-stroke-data.csv
#The author of the dataset is fedesoriano

# Load the registered dataset from workspace
dataset = Dataset.get_by_name(ws, name='healthcare-dataset-stroke-data')

# from azureml.data.dataset_factory import TabularDatasetFactory
# from azureml.core.dataset import Dataset

# url_paths = [
#     "https://www.kaggle.com/fedesoriano/stroke-prediction-dataset/download"
#     ]

# # dataset = TabularDatasetFactory.from_delimited_files(path=url_paths)
# dataset = Dataset.Tabular.from_delimited_files(path=url_paths)
# type(dataset)

# Convert the dataset to dataframe
df = dataset.to_pandas_dataframe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
gender               5110 non-null object
age                  5110 non-null float64
hypertension         5110 non-null bool
heart_disease        5110 non-null bool
ever_married         5110 non-null bool
work_type            5110 non-null object
Residence_type       5110 non-null object
avg_glucose_level    5110 non-null float64
bmi                  4909 non-null float64
smoking_status       5110 non-null object
stroke               5110 non-null bool
dtypes: bool(4), float64(3), object(4)
memory usage: 299.5+ KB


In [6]:
df.describe()

,age,avg_glucose_level,bmi
count,5110.000000,5110.000000,4909.000000
mean,43.226614,106.147677,28.893237
std,22.612647,45.283560,7.854067
min,0.080000,55.120000,10.300000
25%,25.000000,77.245000,23.500000
50%,45.000000,91.885000,28.100000
75%,61.000000,114.090000,33.100000
max,82.000000,271.740000,97.600000


In [7]:
df.head(10)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,False,True,True,Private,Urban,228.69,36.6,formerly smoked,True
1,Female,61.0,False,False,True,Self-employed,Rural,202.21,NaN,never smoked,True
2,Male,80.0,False,True,True,Private,Rural,105.92,32.5,never smoked,True
3,Female,49.0,False,False,True,Private,Urban,171.23,34.4,smokes,True
4,Female,79.0,True,False,True,Self-employed,Rural,174.12,24.0,never smoked,True
5,Male,81.0,False,False,True,Private,Urban,186.21,29.0,formerly smoked,True
6,Male,74.0,True,True,True,Private,Rural,70.09,27.4,never smoked,True
7,Female,69.0,False,False,False,Private,Urban,94.39,22.8,never smoked,True
8,Female,59.0,False,False,True,Private,Rural,76.15,NaN,Unknown,True
9,Female,78.0,False,False,True,Private,Urban,58.57,24.2,Unknown,True


In [8]:
# Create the CPU computer cluster
amlcompute_cluster_name = "capstone"

# Verify if cluster does not exist otherwise use the existing one
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=5,
                                                           min_nodes=0,
                                                           idle_seconds_before_scaledown=600)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

Overview of the AutoML settings and the configuration that was used:
- "experiment_timeout_minutes": set to 30 minutes. The experiment will timeout after after this period of time to avoid incurring additional expense.
- "max_concurrent_iterations": set to 4. The max number of concurrent iterations that can be run on the nodes in the compute cluster.
- "primary_metric" : set to 'accuracy' because this is a classification problems.
- "n_cross_validations": set to 5, therefore the training and validation sets will be divided into five equal sets.
- "iterations": set to 25. 
- compute_target: set to the CPU compute cluster name defined above.
- task: set to 'classification' because our target is to predict whether the patient had a stroke or not.
- training_data: the dataset for the project.
- label_column_name: set to the target colunm in the dataset 'stroke' (true or false).
- enable_early_stopping: enabled to terminate the experiment if the accuracy score does not improveme over time, thus avoiding unnecessary costs.
- featurization = set to 'auto' so Azure ML will automatically handle featurization and clean the dataset. 
- debug_log: errors will be logged into 'automl_errors.log'.

In [9]:


# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5,
    "iterations": 25
    }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_target,
    task = 'classification',
    training_data=dataset,
    label_column_name='stroke',
    enable_early_stopping= True,
    featurization = 'auto',
    debug_log = 'automl_errors.log',
    **automl_settings)


In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on capstone with default configuration
Running on remote compute: capstone


Experiment,Id,Type,Status,Details Page,Docs Page
automl-stroke,AutoML_4ab0a4e3-709f-4fe5-90bb-e237e0f21eda,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=====

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [12]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl-stroke,AutoML_4ab0a4e3-709f-4fe5-90bb-e237e0f21eda,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|249                              |True                             |5110                                  |
+---------------------------------+----------

{'runId': 'AutoML_4ab0a4e3-709f-4fe5-90bb-e237e0f21eda',
 'target': 'capstone',
 'status': 'Completed',
 'startTimeUtc': '2021-09-23T05:01:42.075891Z',
 'endTimeUtc': '2021-09-23T05:19:30.206177Z',
 'properties': {'num_iterations': '25',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'capstone',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-stroke","subscription_id":"bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8","resource_group":"udacity-nano-degree","workspace_name":"udacity-courtlin","region":"eastus2","compute_target":"capstone","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":25,"primary_metric":"accuracy","task_type":"classification","d

In [13]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-stroke,AutoML_4ab0a4e3-709f-4fe5-90bb-e237e0f21eda,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
# Retrieve and save the best automl model
best_model, model = remote_run.get_output()
print(best_model)
print(model.steps)

Package:azureml-automl-runtime, training version:1.34.0, current version:1.33.0
Package:azureml-core, training version:1.34.0, current version:1.33.0
Package:azureml-dataprep, training version:2.22.2, current version:2.20.1
Package:azureml-dataprep-rslex, training version:1.20.2, current version:1.18.0
Package:azureml-dataset-runtime, training version:1.34.0, current version:1.33.0
Package:azureml-defaults, training version:1.34.0, current version:1.33.0
Package:azureml-interpret, training version:1.34.0, current version:1.33.0
Package:azureml-mlflow, training version:1.34.0, current version:1.33.0
Package:azureml-pipeline-core, training version:1.34.0, current version:1.33.0
Package:azureml-responsibleai, training version:1.34.0, current version:1.33.0
Package:azureml-telemetry, training version:1.34.0, current version:1.33.0
Package:azureml-train-automl-client, training version:1.34.0, current version:1.33.0
Package:azureml-train-automl-runtime, training version:1.34.0, current versi

Run(Experiment: automl-stroke,
Id: AutoML_4ab0a4e3-709f-4fe5-90bb-e237e0f21eda_23,
Type: azureml.scriptrun,
Status: Completed)
[('datatransformer', DataTransformer(
    task='classification',
    is_onnx_compatible=False,
    enable_feature_sweeping=True,
    enable_dnn=False,
    force_text_dnn=False,
    feature_sweeping_timeout=86400,
    featurization_config=None,
    is_cross_validation=True,
    feature_sweeping_config={}
)), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(
    estimators=[('18', Pipeline(
        memory=None,
        steps=[('standardscalerwrapper', StandardScalerWrapper(
            copy=True,
            with_mean=False,
            with_std=False
        )), ('xgboostclassifier', XGBoostClassifier(
            random_state=0,
            n_jobs=1,
            problem_info=ProblemInfo(
                gpu_training_param_dict={'processing_unit_type': 'cpu'}
            ),
            booster='gbtree',
            colsample_bytree=0.7,
          

In [15]:
print(model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=1.3541666666666667, reg_lambda=1.4583333333333335, subsample=1, tree_method='auto'))], verbose=False)), ('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(), random_state=None))], verbose=False)), ('13', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('sgdclassifierwrapper', SGDClassifierWrapper(alpha=7.5510448979591835, class_weight='balanced', eta0=0.001, fit_intercept=True, l1_ratio=0.42857142857142855, learning_rate='cons

In [16]:
remote_run.get_metrics()

{'experiment_status': ['DatasetEvaluation',
  'FeaturesGeneration',
  'DatasetFeaturization',
  'DatasetFeaturizationCompleted',
  'DatasetBalancing',
  'DatasetCrossValidationSplit',
  'ModelSelection',
  'BestRunExplainModel',
  'ModelExplanationDataSetSetup',
  'PickSurrogateModel',
  'EngineeredFeatureExplanations',
  'EngineeredFeatureExplanations',
  'RawFeaturesExplanations',
  'RawFeaturesExplanations',
  'BestRunExplainModel'],
 'experiment_status_description': ['Gathering dataset statistics.',
  'Generating features for the dataset.',
  'Beginning to fit featurizers and featurize the dataset.',
  'Completed fit featurizers and featurizing the dataset.',
  'Performing class balancing sweeping',
  'Generating individually featurized CV splits.',
  'Beginning model selection.',
  'Best run model explanations started',
  'Model explanations data setup completed',
  'Choosing LightGBM as the surrogate model for explanations',
  'Computation of engineered features started',
  'Comp

In [17]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [18]:
# Register the best model with Azure
model = best_model.register_model(model_name='automl-best-stroke-model',model_path='outputs/model.pkl')

In [28]:
# Get the AutoML environment details with its dependencies
environment = Environment.get(ws, "AzureML-AutoML")

In [29]:
print(environment)

Environment(Name: AzureML-AutoML,
Version: 83)


In [30]:
inference_config = InferenceConfig(entry_script='scoring.py',
                                   environment=environment)
service_name = 'automl-best-stroke-deployed'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True, auth_enabled=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,                       
                       overwrite=True
                      )
service.wait_for_deployment(show_output=True)

scoring_uri = service.scoring_uri
print(scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-23 05:29:06+00:00 Creating Container Registry if not exists.
2021-09-23 05:29:06+00:00 Registering the environment.
2021-09-23 05:29:07+00:00 Use the existing image.
2021-09-23 05:29:07+00:00 Generating deployment configuration.
2021-09-23 05:29:08+00:00 Submitting deployment to compute.
2021-09-23 05:29:12+00:00 Checking the status of deployment automl-best-stroke-deployed..
2021-09-23 05:32:37+00:00 Checking the status of inference endpoint automl-best-stroke-deployed.
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://f515466b-999c-4e85-87d7-89a671163cb9.eastus2.azurecontainer.io/score


In [31]:
# Enable app insights
# service.update(enable_app_insights=True)

In [32]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

TODO: In the cell below, send a request to the web service you deployed to test it.

In [33]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')




service state: Healthy

scoring URI: 
http://f515466b-999c-4e85-87d7-89a671163cb9.eastus2.azurecontainer.io/score

swagger URI: 
http://f515466b-999c-4e85-87d7-89a671163cb9.eastus2.azurecontainer.io/swagger.json



In [36]:
import requests

key = "I7ZPDaYLEdGhsjFA3Mu3yk698fjizohD"
data = {
  "data": [
    {
      "gender": "Male",
      "age": 67,
      "hypertension": "False",
      "heart_disease": "True",
      "ever_married": "True",
      "work_type": "Private",
      "Residence_type": "Urban",
      "avg_glucose_level": 228.69,
      "bmi": 36.6,
      "smoking_status": "formerly smoked"
    },
    {
      "gender": "Female",
      "age": 50,
      "hypertension": "False",
      "heart_disease": "False",
      "ever_married": "False",
      "work_type": "0.0",
      "Residence_type": "0.0",
      "avg_glucose_level": 0,
      "bmi": 0,
      "smoking_status": "0"
    }
  ]
}

# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {"Content-Type": "application/json"}
# If authentication is enabled, set the authorization header
headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())


{"result": [false, false]}


In [40]:
# Sample code from Azure -> Endpoints -> CONSUME tab
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
data = {
    "data":
    [
        {
            'gender': "Male",
            'age': "100",
            'hypertension': "True",
            'heart_disease': "True",
            'ever_married': "True",
            'work_type': "Govt_job",
            'Residence_type': "Urban",
            'avg_glucose_level': "200",
            'bmi': "50",
            'smoking_status': "smoking",
        },

        {
            'gender': "Male",
            'age': "100",
            'hypertension': "True",
            'heart_disease': "True",
            'ever_married': "True",
            'work_type': "Govt_job",
            'Residence_type': "Urban",
            'avg_glucose_level': "200",
            'bmi': "50",
            'smoking_status': "smoking",
        }
    ],
}

body = str.encode(json.dumps(data))

url = 'http://f515466b-999c-4e85-87d7-89a671163cb9.eastus2.azurecontainer.io/score'
api_key = 'I7ZPDaYLEdGhsjFA3Mu3yk698fjizohD' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))


b'"{\\"result\\": [false, false]}"'


In [42]:
# #Test response using the endpoint.py file
# %run endpoint.py

TODO: In the cell below, print the logs of the web service and delete the service

In [41]:
#Print the logs from the service
logs = service.get_logs()
logs

'2021-09-23T05:32:20,715671000+00:00 - gunicorn/run \nDynamic Python package installation is disabled.\nStarting HTTP server\n2021-09-23T05:32:20,734344000+00:00 - iot-server/run \n2021-09-23T05:32:20,735767200+00:00 - rsyslog/run \n2021-09-23T05:32:20,746703600+00:00 - nginx/run \nrsyslogd: /azureml-envs/azureml_732632de80c129004c31329a710852e7/lib/libuuid.so.1: no version information available (required by rsyslogd)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2021-09-23T05:32:21,091851300+00:00 - iot-server/finish 1 0\n2021-09-23T05:32:21,098957000+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (70)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 108\nSPARK_HOME not set. Skipping PySpark Initialization.\nGenerating new fontManager, this may take some time...\nInitializing logger\n2021-09-23 05:32:26,266 | root | INFO | Starting up app insights client

In [ ]:
# Delete the deployed model
service.delete()

#Delete the CPU compute target
compute_target.delete()